# Support Vector Regression

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# From: https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8
import timesplit as ts

## Set up helper functions

In [2]:
def GridSearchSummary(search, hyperparams, metrics, file):
    """
    Creates a dataframe with summary results from GridSearchCV
    search - the final GridSearchCV object
    hyperparams - the list of parameters you were tuning for (aka keys in the param_grid dictionary)
    metrics - the list of scoring metrics you used
    file - filename of where you want to save your results
    """
    
    summary = pd.DataFrame()
    
    for hyperparam in hyperparams:
        col = [dct[hyperparam] for dct in search.cv_results_["params"]]
        summary[hyperparam] = col
    
    for metric in metrics:
        col = search.cv_results_["mean_test_" + metric]
        summary[metric] = col
    
    summary.to_csv(file)
    
    return summary

## Import data

In [3]:
X_train = pd.read_csv("../Data/Train-Test Set/X_train.csv", parse_dates=["date"])
y_train = pd.read_csv("../Data/Train-Test Set/y_train.csv")

X_test = pd.read_csv("../Data/Train-Test Set/X_test.csv", parse_dates=["date"])
y_test = pd.read_csv("../Data/Train-Test Set/y_test.csv")

## Get indices for time-based cross validation

In [4]:
# set up time-based CV indices
tscv = ts.TimeBasedCV(train_period=21,
                      test_period=7,
                      freq='days')

tscv_indices = []

for train_index, test_index in tscv.split(X_train, date_column='date'):
    tscv_indices.append((train_index, test_index))

## Try adding some polynomial features

In [5]:
from sklearn.preprocessing import PolynomialFeatures

poly_vars = ["new_cases_7avg", "2weeksago_cases_7avg", "new_cases", "cumulative_cases", 
             "prev_day_adult_admit_7daysum", "prev_day_adult_admit_80+_7daysum", 
             "prev_day_adult_admit_50-59_7daysum"]

for var in poly_vars:
    X_train[var + "_2"] = X_train[var]**2
    X_test[var + "_2"] = X_test[var]**2

## Run GridSearchCV using Support Vector Regression

In [6]:
# define scoring metrics
scoring_metrics = ["neg_mean_squared_error", "r2", "neg_mean_absolute_error"]

In [8]:
param_grid = {
    'kernel': ["linear", "rbf"],
    'C': [0.5, 1, 5, 10, 20],
    'epsilon': [0, 5, 10]
}

print("Start: ", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

SVR_search_pt1 = GridSearchCV(svm.SVR(), param_grid, cv=tscv_indices, scoring=scoring_metrics, refit = False, verbose=2)
SVR_search_pt1.fit(X_train.drop(columns=["date"]), np.array(y_train).ravel())

print("End: ", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Start:  29/05/2021 22:48:27
Fitting 39 folds for each of 30 candidates, totalling 1170 fits
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.3s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.4s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.2s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.2s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.3s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.3s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.2s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.2s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.2s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.2s
[CV] END ....................C=0.5, epsilon=0, kernel=linear; total time=   1.4s
[CV] END ........

[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.7s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.4s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.5s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.4s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.3s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.3s
[CV] END ....................C=0.5, epsilon=5, kernel=linear; total time=   0.3s
[CV] END ...................

[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   0.5s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   0.7s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   0.9s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.3s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.5s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.5s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.7s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.8s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.7s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.7s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.5s
[CV] END ......................C=0.5, epsilon=10, kernel=rbf; total time=   1.5s
[CV] END ...................

[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END .........................C=1, epsilon=0, kernel=rbf; total time=   2.1s
[CV] END ......................C=1, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ......................C=1, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ......................C=1, epsilon=5, kernel=linear; total time=   0.2s
[CV] END ......................C=1, epsilon=5, kernel=linear; total time=   0.3s
[CV] END ......................C=1, epsilon=5, kernel=linear; total time=   0.3s
[CV] END ...................

[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   1.0s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.7s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.8s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.7s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.6s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.8s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.5s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.4s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.4s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.3s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.2s
[CV] END .....................C=1, epsilon=10, kernel=linear; total time=   0.1s
[CV] END ...................

...

[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   4.6s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   5.0s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   8.4s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   9.7s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   9.9s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   7.4s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=  14.0s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   9.9s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=  11.8s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   9.1s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=  12.2s
[CV] END .....................C=10, epsilon=0, kernel=linear; total time=   8.4s
[CV] END ...................

[CV] END .....................C=10, epsilon=5, kernel=linear; total time=   1.2s
[CV] END .....................C=10, epsilon=5, kernel=linear; total time=   0.6s
[CV] END .....................C=10, epsilon=5, kernel=linear; total time=   0.8s
[CV] END .....................C=10, epsilon=5, kernel=linear; total time=   0.3s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.4s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.4s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.4s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.4s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.5s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.5s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.5s
[CV] END ........................C=10, epsilon=5, kernel=rbf; total time=   0.5s
[CV] END ...................

[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   1.1s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   1.1s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   1.0s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.8s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.6s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.5s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.4s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.3s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.2s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.2s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.2s
[CV] END .......................C=10, epsilon=10, kernel=rbf; total time=   0.2s
[CV] END ...................

[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   0.8s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   0.5s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   0.5s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   0.7s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   1.0s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   2.3s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   3.2s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   3.9s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   7.7s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   8.0s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   7.2s
[CV] END .....................C=20, epsilon=5, kernel=linear; total time=   7.5s
[CV] END ...................

[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.5s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.6s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.8s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.6s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.6s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.6s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.9s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.3s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.7s
[CV] END ....................C=20, epsilon=10, kernel=linear; total time=   0.4s
[CV] END .......................C=20, epsilon=10, kernel=rbf; total time=   0.2s
[CV] END .......................C=20, epsilon=10, kernel=rbf; total time=   0.2s
[CV] END ...................

In [9]:
SVR_summary_pt1 = GridSearchSummary(SVR_search_pt1, 
                  ["kernel", "C", "epsilon"],
                  scoring_metrics,
                  "../Data/Model Results/SVR_search_pt1.csv")

SVR_summary_pt1.sort_values(by=["neg_mean_absolute_error"])

,kernel,C,epsilon,neg_mean_squared_error,r2,neg_mean_absolute_error
5,rbf,0.5,10,-16734.416843,0.142110,-26.745321
3,rbf,0.5,5,-16784.707850,0.140646,-25.221659
11,rbf,1.0,10,-15864.307330,0.217003,-25.061725
1,rbf,0.5,0,-16812.878683,0.137167,-24.635523
9,rbf,1.0,5,-15894.814419,0.218874,-23.413158
7,rbf,1.0,0,-15921.075653,0.217120,-22.745592
17,rbf,5.0,10,-12577.828614,0.377521,-19.877095
15,rbf,5.0,5,-12594.855411,0.380598,-18.070914
23,rbf,10.0,10,-11086.013754,0.439059,-17.727779
13,rbf,5.0,0,-12605.987581,0.379165,-17.260574


In [11]:
param_grid = {
    'kernel': ["poly"],
    'degree': [2, 3],
    'C': [0.5, 1, 5, 10, 20],
    'epsilon': [0, 5, 10]
}

print("Start: ", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

SVR_search_pt2 = GridSearchCV(svm.SVR(), param_grid, cv=tscv_indices, scoring=scoring_metrics, refit = False, verbose=2)
SVR_search_pt2.fit(X_train.drop(columns=["date"]), np.array(y_train).ravel())

print("End: ", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Start:  29/05/2021 23:26:18
Fitting 39 folds for each of 30 candidates, totalling 1170 fits
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.3s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.3s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.2s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.4s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.3s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.5s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.3s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.2s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.3s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.3s
[CV] END ............C=0.5, degree=2, epsilon=0, kernel=poly; total time=   1.4s
[CV] END ........

[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.8s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.6s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.5s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.4s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.5s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.6s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.9s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   0.9s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   1.4s
[CV] END ...........C=0.5, degree=2, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ...........C=0.5, d

[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   0.6s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   0.7s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   0.8s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   0.9s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.8s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.1s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.8s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.5s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.4s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   1.3s
[CV] END ...........C=0.5, degree=3, epsilon=10, kernel=poly; total time=   0.9s
[CV] END ...........C=0.5, d

[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.7s
[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.9s
[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.8s
[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.8s
[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.7s
[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.7s
[CV] END ..............C=1, degree=2, epsilon=5, kernel=poly; total time=   0.6s
[CV] END .............C=1, degree=2, epsilon=10, kernel=poly; total time=   0.3s
[CV] END .............C=1, degree=2, epsilon=10, kernel=poly; total time=   0.4s
[CV] END .............C=1, degree=2, epsilon=10, kernel=poly; total time=   0.3s
[CV] END .............C=1, degree=2, epsilon=10, kernel=poly; total time=   0.4s
[CV] END .............C=1, degree=2, epsilon=10, kernel=poly; total time=   0.6s
[CV] END .............C=1, d

[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.3s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.4s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.5s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.4s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.9s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.7s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.1s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.2s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.1s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   2.0s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.1s
[CV] END ..............C=1, degree=3, epsilon=5, kernel=poly; total time=   1.0s
[CV] END ..............C=1, 

...

[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.5s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   2.3s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.8s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.5s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.5s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.5s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   2.9s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.4s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.6s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.5s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.4s
[CV] END .............C=10, degree=2, epsilon=0, kernel=poly; total time=   1.6s
[CV] END .............C=10, 

[CV] END ............C=10, degree=2, epsilon=10, kernel=poly; total time=   0.5s
[CV] END ............C=10, degree=2, epsilon=10, kernel=poly; total time=   0.6s
[CV] END ............C=10, degree=2, epsilon=10, kernel=poly; total time=   0.5s
[CV] END ............C=10, degree=2, epsilon=10, kernel=poly; total time=   0.5s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   2.2s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   2.0s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   1.6s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   1.8s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   2.2s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   1.7s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   1.9s
[CV] END .............C=10, degree=3, epsilon=0, kernel=poly; total time=   1.7s
[CV] END .............C=10, 

[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   2.1s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   3.2s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   2.1s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   2.2s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   1.5s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   2.0s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   1.6s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   0.6s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   1.2s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   0.9s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   0.7s
[CV] END ............C=10, degree=3, epsilon=10, kernel=poly; total time=   1.1s
[CV] END ............C=10, d

[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   0.4s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   0.4s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   0.7s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   0.9s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.9s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.4s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.0s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.1s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.4s
[CV] END ............C=20, degree=2, epsilon=10, kernel=poly; total time=   1.2s
[CV] END ............C=20, d

[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   0.8s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.0s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.0s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.6s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.6s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.3s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.6s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.1s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.0s
[CV] END .............C=20, degree=3, epsilon=5, kernel=poly; total time=   1.2s
[CV] END ............C=20, degree=3, epsilon=10, kernel=poly; total time=   0.8s
[CV] END ............C=20, degree=3, epsilon=10, kernel=poly; total time=   0.7s
[CV] END ............C=20, d

In [13]:
SVR_summary_pt2 = GridSearchSummary(SVR_search_pt2, 
                  ["kernel", "degree", "C", "epsilon"],
                  scoring_metrics,
                  "../Data/Model Results/SVR_search_pt2.csv")

SVR_summary_pt2.sort_values(by=["neg_mean_absolute_error"])

,kernel,degree,C,epsilon,neg_mean_squared_error,r2,neg_mean_absolute_error
21,poly,3,10.0,0,-1.186306e+09,-76436.845044,-457.158649
27,poly,3,20.0,0,-9.571527e+08,-63832.168235,-448.604583
28,poly,3,20.0,5,-6.411724e+08,-42495.015733,-385.076167
15,poly,3,5.0,0,-8.344723e+08,-49587.595049,-378.387293
22,poly,3,10.0,5,-7.070966e+08,-43687.177207,-370.085190
16,poly,3,5.0,5,-6.346857e+08,-37288.831154,-338.041323
23,poly,3,10.0,10,-4.474959e+08,-25511.426375,-307.579378
17,poly,3,5.0,10,-4.308039e+08,-24140.866814,-288.182284
29,poly,3,20.0,10,-2.077796e+08,-12464.580838,-261.701589
9,poly,3,1.0,0,-3.637038e+07,-955.544034,-90.915043


In [18]:
SVR_summary = pd.concat([SVR_summary_pt1, SVR_summary_pt2])
SVR_summary.sort_values(by=["neg_mean_absolute_error"], ascending = False).head()

,kernel,C,epsilon,neg_mean_squared_error,r2,neg_mean_absolute_error,degree
18,linear,10.0,0,-4056.807724,0.879607,-7.668122,NaN
12,linear,5.0,0,-4078.928718,0.882553,-7.684356,NaN
24,linear,20.0,0,-4085.938540,0.878131,-7.687117,NaN
6,linear,1.0,0,-3911.249237,0.890480,-7.756379,NaN
0,linear,0.5,0,-4142.278106,0.890822,-7.929125,NaN


## Run GridSearchCV using LinearSVR

In [19]:
# define scoring metrics
scoring_metrics = ["neg_mean_squared_error", "r2", "neg_mean_absolute_error"]

In [21]:
from sklearn.pipeline import make_pipeline

param_grid = {
    'C': [0.1, 0.5, 1, 5, 10, 20],
    'epsilon': [0, 5, 10]
}

print("Start: ", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

linSVR_search = GridSearchCV(svm.LinearSVR(max_iter=100000), param_grid, cv=tscv_indices, 
                             scoring=scoring_metrics, refit = False, verbose=2)
linSVR_search.fit(X_train.drop(columns=["date"]), np.array(y_train).ravel())

print("End: ", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Start:  29/05/2021 23:54:09
Fitting 39 folds for each of 18 candidates, totalling 702 fits
[CV] END ...................................C=0.1, epsilon=0; total time=   0.1s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.1s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.0s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.1s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.0s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.0s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.0s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.0s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.0s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.1s
[CV] END ...................................C=0.1, epsilon=0; total time=   0.4s
[CV] END .........

[CV] END ..................................C=0.1, epsilon=10; total time=   0.8s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.6s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.6s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.3s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.4s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.4s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.3s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.3s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.3s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.4s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.3s
[CV] END ..................................C=0.1, epsilon=10; total time=   0.4s
[CV] END ...................

/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] END ...................................C=0.5, epsilon=0; total time=  22.0s
[CV] END ...................................C=0.5, epsilon=0; total time=  14.2s


/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] END ...................................C=20, epsilon=10; total time=  23.4s


/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] END ...................................C=20, epsilon=10; total time=  23.4s
[CV] END ...................................C=20, epsilon=10; total time=  23.0s
End:  30/05/2021 02:06:13


/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Some of the above specifications do not converge well...

In [22]:
linSVR_summary = GridSearchSummary(linSVR_search, 
                  ["epsilon", "C"],
                  scoring_metrics,
                  "../Data/Model Results/linSVR_search.csv")

linSVR_summary.sort_values(by=["neg_mean_absolute_error"])

,epsilon,C,neg_mean_squared_error,r2,neg_mean_absolute_error
2,10,0.1,-2915.588096,0.904891,-9.716589
5,10,0.5,-4104.978040,0.889098,-9.013524
8,10,1.0,-3937.937377,0.889712,-8.921313
1,5,0.1,-2773.122051,0.910023,-8.851271
14,10,10.0,-4377.724809,0.874364,-8.827996
11,10,5.0,-4130.325085,0.882778,-8.809221
17,10,20.0,-4254.240960,0.876739,-8.798473
0,0,0.1,-2735.567145,0.910665,-8.502936
4,5,0.5,-4052.564126,0.891217,-8.310909
7,5,1.0,-3922.484399,0.892655,-8.148442
